<a href="https://colab.research.google.com/github/gunjak/Placement-Assignment_Gunja/blob/main/DL_ques2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import torch
import torchvision
import tarfile
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split
from torch import nn 
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data.dataloader import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset
import random
torch.manual_seed(0)
dataset_url = "https://s3.amazonaws.com/fast-ai-imageclas/cifar10.tgz"
download_url(dataset_url, '.')

100%|██████████| 135107811/135107811 [00:03<00:00, 37595501.31it/s]


In [ ]:
from tqdm.auto import tqdm

In [ ]:
!tar -xvzf /content/cifar10.tgz

In [ ]:
import cv2
image=cv2.imread('/content/cifar10/test/airplane/0001.png')
image.shape

(32, 32, 3)

In [ ]:
train_dir='/content/cifar10/train'
test_dir='/content/cifar10/test'

In [ ]:
device=torch.device('cuda')

In [ ]:
import glob
classes=['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
count=0
train_files=[]
while count!=10:
  for name in glob.glob(f'{train_dir}/{classes[count]}/*'):
    train_files.append(name)
  count+=1
test_files=[]
count=0
while count!=10:
  for name in glob.glob(f'{test_dir}/{classes[count]}/*'):
    test_files.append(name)
  count+=1


In [ ]:
random.shuffle(train_files)

In [ ]:
class CifarDataset(Dataset):
    def __init__(self, images_filepaths, transform=None):
        self.images_filepaths = images_filepaths
        self.transform = transform

    def __len__(self):
        return len(self.images_filepaths)

    def __getitem__(self, idx):
        image_filepath = self.images_filepaths[idx]
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if os.path.normpath(image_filepath).split(os.sep)[-2] == classes[0]:
            label = 0.0
        elif os.path.normpath(image_filepath).split(os.sep)[-2] == classes[1]:
            label = 1.0
        elif os.path.normpath(image_filepath).split(os.sep)[-2] == classes[2]:
            label = 2.0
        elif os.path.normpath(image_filepath).split(os.sep)[-2] == classes[3]:
            label = 3.0
        elif os.path.normpath(image_filepath).split(os.sep)[-2] == classes[4]:
            label = 4.0
        elif os.path.normpath(image_filepath).split(os.sep)[-2] == classes[5]:
            label = 5.0
        elif os.path.normpath(image_filepath).split(os.sep)[-2] == classes[6]:
            label = 6.0
        elif os.path.normpath(image_filepath).split(os.sep)[-2] == classes[7]:
            label = 7.0
        elif os.path.normpath(image_filepath).split(os.sep)[-2] == classes[8]:
            label = 8.0
        elif os.path.normpath(image_filepath).split(os.sep)[-2] == classes[9]:
            label = 9.0
        if self.transform is not None:
            image = self.transform(image=image)["image"]
        return image, label
correct_images_filepaths = [i for i in train_files if cv2.imread(i) is not None]


In [ ]:
train_transform = A.Compose(
    [
        A.SmallestMaxSize(max_size=160),
        A.RandomCrop(height=128, width=128),
        A.RandomBrightnessContrast(p=0.5),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ]
)

In [ ]:
train_dataset = CifarDataset(images_filepaths=train_files, transform=train_transform)
test_dataset=CifarDataset(images_filepaths=test_files, transform=train_transform)

In [ ]:
val_size = 5000
train_size = len(train_dataset) - val_size

train_ds, val_ds = random_split(train_dataset, [train_size, val_size])
len(train_ds), len(val_ds)

(45000, 5000)

In [ ]:
params = {
    "num_workers": 4}
NUM_EPOCHS = 5
BATCH_SIZE = 10
LEARNING_RATE = 0.001

In [ ]:
train_dl = DataLoader(
    train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=params["num_workers"], pin_memory=True,
)
val_dl=DataLoader(val_ds,batch_size=BATCH_SIZE, shuffle=True, num_workers=params["num_workers"], pin_memory=True,)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
class TinyVGG(nn.Module):
   
    def __init__(self, input_shape: int,output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
          nn.Conv2d(in_channels=input_shape, 
                    out_channels=16, 
                    kernel_size=3, 
                    stride=1, 
                    padding=0),  
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,
                        stride=2)
        )
        self.conv_block_2 = nn.Sequential(
          nn.Conv2d(16,16, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.MaxPool2d(2)

        )
        self.classifier = nn.Sequential(
          nn.Flatten(),
          nn.LazyLinear(
                    out_features=output_shape)
        )
    
    def forward(self, x: torch.Tensor):
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        x = self.classifier(x)
        return x

In [ ]:
class TinyVGG1(nn.Module):
   
    def __init__(self, input_shape: int,output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
          nn.Conv2d(in_channels=input_shape, 
                    out_channels=16, 
                    kernel_size=5, 
                    stride=2, 
                    padding=0),  
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,
                        stride=1))
        
        self.conv_block_2 = nn.Sequential(
          nn.Conv2d(16,16, kernel_size=3, padding=0),
          nn.ReLU(),
          nn.MaxPool2d(2,stride=2)

        )
        
        self.classifier = nn.Sequential(
          nn.Flatten(),
          nn.LazyLinear(
                    out_features=output_shape)
        )
        
    
    def forward(self, x: torch.Tensor):
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        x = self.classifier(x)
        return x

In [ ]:
class TinyVGG2(nn.Module):
    def __init__(self, input_shape: int,output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
          nn.Conv2d(in_channels=input_shape, 
                    out_channels=16, 
                    kernel_size=3, 
                    stride=1, 
                    padding=0),  
          nn.ReLU(),
          nn.Conv2d(in_channels=16, 
                    out_channels=16,
                    kernel_size=3,
                    stride=2,
                    padding=0),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2)
        )
        self.classifier = nn.Sequential(
          nn.Flatten(),
          nn.LazyLinear(
                    out_features=output_shape)
        )
    def forward(self, x: torch.Tensor):
        x = self.conv_block_1(x)
        x = self.classifier(x)
        return x

In [ ]:

class TinyVGG3(nn.Module):
    def __init__(self, input_shape: int,output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
          nn.Conv2d(in_channels=input_shape, 
                    out_channels=32, 
                    kernel_size=5, 
                    stride=2, 
                    padding=0),  
          nn.ReLU(),
          nn.Conv2d(in_channels=32, 
                    out_channels=16,
                    kernel_size=3,
                    stride=2,
                    padding=0),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2)
        )
        self.classifier = nn.Sequential(
          nn.Flatten(),
          nn.LazyLinear(
                    out_features=output_shape)
        )
    def forward(self, x: torch.Tensor):
        x = self.conv_block_1(x)
        x = self.classifier(x)
        return x

In [ ]:
class TinyVGG4(nn.Module):
   
    def __init__(self, input_shape: int,output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
          nn.Conv2d(in_channels=input_shape, 
                    out_channels=16, 
                    kernel_size=5, 
                    stride=2, 
                    padding=0),  
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2,
                        stride=2),
          nn.Sequential(
          nn.Conv2d(16,16, kernel_size=5, padding=0),
          nn.ReLU(),
          nn.MaxPool2d(2,stride=2))
        )
        self.classifier = nn.Sequential(
          nn.Flatten(),
          nn.LazyLinear(
                    out_features=output_shape)
        )
    
    def forward(self, x: torch.Tensor):
        x = self.conv_block_1(x)
        x = self.classifier(x)
        return x

In [ ]:
from torchsummary import summary
model1=TinyVGG(3,10).to(device)
summary(model1,(3, 32, 32))
model2=TinyVGG1(3,10).to(device)
summary(model2,(3, 32,32))
model3=TinyVGG2(3,10).to(device)
summary(model3,(3, 32, 32))
model4=TinyVGG3(3,10).to(device)
summary(model4,(3, 32, 32))
model5=TinyVGG4(3,10).to(device)
summary(model5,(3, 32, 32))

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 30, 30]             448
              ReLU-2           [-1, 16, 30, 30]               0
         MaxPool2d-3           [-1, 16, 15, 15]               0
            Conv2d-4           [-1, 16, 13, 13]           2,320
              ReLU-5           [-1, 16, 13, 13]               0
         MaxPool2d-6             [-1, 16, 6, 6]               0
           Flatten-7                  [-1, 576]               0
            Linear-8                   [-1, 10]           5,770
Total params: 8,538
Trainable params: 8,538
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.30
Params size (MB): 0.03
Estimated Total Size (MB): 0.34
----------------------------------------------------------------
----------------------------------------------

In [ ]:

model=TinyVGG(
    input_shape=3,
    output_shape=len(classes)
)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=LEARNING_RATE)


In [ ]:
model1=TinyVGG(
    input_shape=3,
    output_shape=len(classes)
).to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model1.parameters(),
                             lr=LEARNING_RATE)
model2=TinyVGG(
    input_shape=3,
    output_shape=len(classes)
).to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.parameters(),
                             lr=LEARNING_RATE)
model3=TinyVGG(
    input_shape=3,
    output_shape=len(classes)
).to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model3.parameters(),
                             lr=LEARNING_RATE)
model4=TinyVGG(
    input_shape=3,
    output_shape=len(classes)
).to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model4.parameters(),
                             lr=LEARNING_RATE)
model5=TinyVGG(
    input_shape=3,
    output_shape=len(classes)
).to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model5.parameters(),
                             lr=LEARNING_RATE)


In [ ]:
def train(model):
  results = {"train_loss": [],
                "train_acc": [],
                "test_loss": [],
                "test_acc": []
      }
  for epoch in tqdm(range(NUM_EPOCHS)):  
    model.train()
    train_loss, train_acc = 0, 0
    for batch, (X, y) in enumerate(train_dl):
      y=y.type(torch.LongTensor)
      X, y = X.to(device), y.to(device)
            # 1. Forward pass
      y_pred = model(X)
      
            # 2. Calculate  and accumulate loss
      loss = loss_fn(y_pred, y)
      train_loss += loss.item() 

            # 3. Optimizer zero grad
      optimizer.zero_grad()

            # 4. Loss backward
      loss.backward()

            # 5. Optimizer step
      optimizer.step()

            # Calculate and accumulate accuracy metric across all batches
      y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
      train_acc += (y_pred_class == y).sum().item()/len(y_pred)

        # Adjust metrics to get average loss and accuracy per batch 
    train_loss = train_loss / len(train_dl)
    train_acc = train_acc / len(train_dl)
    model.eval() 

      # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0

      # Turn on inference context manager
    with torch.inference_mode():
          # Loop through DataLoader batches
      for batch, (X, y) in enumerate(val_dl):
        y=y.type(torch.LongTensor)
        X, y = X.to(device), y.to(device)
              # 1. Forward pass
        test_pred_logits = model(X)

              # 2. Calculate and accumulate loss
        loss = loss_fn(test_pred_logits, y)
        test_loss += loss.item()

              # Calculate and accumulate accuracy
        test_pred_labels = test_pred_logits.argmax(dim=1)
        test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

      # Adjust metrics to get average loss and accuracy per batch 
    test_loss = test_loss / len(val_dl)
    test_acc = test_acc / len(val_dl)

          # Update results dictionary
    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)
  return results

    

In [ ]:
models={'model1':model1,
        'model2':model2,
        'model3':model3,
        'model4':model4,
        'model5':model5}
for model_name,model in models.items():
  result=train(model)
  print(f'{model_name} accuracy {result["test_acc"]}')
  print('--------------------------------------------')

  0%|          | 0/5 [00:00<?, ?it/s]

model1 accuracy [0.10140000000000045, 0.09700000000000047, 0.09680000000000037, 0.10000000000000041, 0.09900000000000046]
--------------------------------------------


  0%|          | 0/5 [00:00<?, ?it/s]

model2 accuracy [0.08760000000000034, 0.09040000000000044, 0.08900000000000036, 0.08980000000000035, 0.08940000000000035]
--------------------------------------------


  0%|          | 0/5 [00:00<?, ?it/s]

model3 accuracy [0.09160000000000036, 0.08980000000000035, 0.09240000000000043, 0.08840000000000038, 0.08600000000000034]
--------------------------------------------


  0%|          | 0/5 [00:00<?, ?it/s]

model4 accuracy [0.09620000000000045, 0.09640000000000037, 0.09680000000000036, 0.09840000000000047, 0.09940000000000039]
--------------------------------------------


  0%|          | 0/5 [00:00<?, ?it/s]

model5 accuracy [0.4320000000000005, 0.5048000000000001, 0.5091999999999998, 0.5299999999999994, 0.5423999999999999]
--------------------------------------------
